In [1]:
import random
import numpy as np
import tensorflow as tf
import os

# Set seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
import os

meta = pd.read_csv('Train_Test_Split.csv')
meta = meta[meta['label'].isin(['AD', 'Healthy'])]

train_meta = meta[meta['split'] == 'train']
test_meta = meta[meta['split'] == 'test']

def load_and_segment(subject_id, data_dir='Data_sampled_128HZ', segment_len=1024):
    file_path = os.path.join(data_dir, f"{subject_id}_data.npy")
    data = np.load(file_path)
    _, time_steps = data.shape
    num_segments = time_steps // segment_len
    if num_segments == 0:
        return np.empty((0, 19, segment_len))
    data = data[:, :num_segments * segment_len]
    segments = data.reshape(19, num_segments, segment_len).transpose(1, 0, 2)
    return segments

def process_data(meta_df, data_dir='Data_sampled_128HZ'):
    X = []
    y = []
    label_map = {'AD': 1, 'Healthy': 0}
    for _, row in meta_df.iterrows():
        segments = load_and_segment(row['subject_id'], data_dir)
        if segments.shape[0] == 0:
            continue
        X.append(segments)
        label = label_map[row['label']]
        one_hot = np.eye(2)[label]
        y.extend([one_hot] * segments.shape[0])
    X = np.concatenate(X, axis=0)
    y = np.array(y)
    return X, y
X_train, y_train = process_data(train_meta)
X_test, y_test = process_data(test_meta)
X_train = (X_train * 1e6) - np.mean(X_train * 1e6, axis=2, keepdims=True)
X_test = (X_test * 1e6) - np.mean(X_test * 1e6, axis=2, keepdims=True)

In [3]:
#CNNSpatial Start to End
X_train_cnn = X_train[..., np.newaxis]  # shape: (N, 19, 1024, 1)
X_test_cnn = X_test[..., np.newaxis]

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization, ReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_eeg_spatial_cnn_seq(input_shape=(19, 1024, 1), dropout_rate=0.3, dense_dim=128):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(dense_dim, activation='relu'),
        Dropout(dropout_rate),

        Dense(2, activation='softmax')
    ])
    return model

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

model = create_eeg_spatial_cnn_seq()
model.compile(
        optimizer=Adam(0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'AUC']
)

model.fit(X_train_cnn, y_train,
          validation_data=(X_test_cnn, y_test),
          epochs=50,
          batch_size=64,
          callbacks=callbacks,
          verbose = 1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 17s 100ms/step - AUC: 0.5445 - accuracy: 0.5340 - loss: 4.4848 - val_AUC: 0.5432 - val_accuracy: 0.5401 - val_loss: 0.6941 - learning_rate: 0.0010
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step - AUC: 0.5763 - accuracy: 0.5530 - loss: 0.6967 - val_AUC: 0.5550 - val_accuracy: 0.5505 - val_loss: 0.6991 - learning_rate: 0.0010
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - AUC: 0.6032 - accuracy: 0.5632 - loss: 0.6748 - val_AUC: 0.6019 - val_accuracy: 0.5884 - val_loss: 0.7419 - learning_rate: 0.0010
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - AUC: 0.7360 - accuracy: 0.6576 - loss: 0.6153 - val_AUC: 0.7282 - val_accuracy: 0.6799 - val_loss: 0.6484 - learning_rate: 0.0010
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 90ms/step - AUC: 0.8191 - accuracy: 0.7103 - loss: 0.5063 - val_AUC: 0.8179 - val_accuracy: 0.7446 - val_loss: 0.5519 - learning_rate: 0.0010
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - AUC: 0.8640 - accuracy: 0.7

In [4]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

y_pred = model.predict(X_test_cnn)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("ROC AUC:", roc_auc_score(y_true, y_pred[:, 1]))
print("Average Precision:", average_precision_score(y_true, y_pred[:, 1]))
print(classification_report(y_true, y_pred_classes))

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
ROC AUC: 0.8945467888809011
Average Precision: 0.9080660519337762
              precision    recall  f1-score   support

           0       0.86      0.67      0.75       535
           1       0.76      0.91      0.83       624

    accuracy                           0.80      1159
   macro avg       0.81      0.79      0.79      1159
weighted avg       0.81      0.80      0.79      1159



In [ ]:
output_folder = "Models"
os.makedirs(output_folder,exist_ok=True)
model_path = os.path.join(output_folder,"Final_CNNSpatial_model.keras")
model.save(model_path)